In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data = pd.read_csv('/kaggle/input/az-handwritten-alphabets-in-csv-format/A_Z Handwritten Data.csv')

In [3]:
data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.639,0.640,0.641,0.642,0.643,0.644,0.645,0.646,0.647,0.648
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
data = np.array(data)

In [5]:
m, n = data.shape
print(data.shape)
print(np.count_nonzero(np.isnan(data)))

(372450, 785)
0


In [6]:
np.random.shuffle(data)
#Seperate into training and testing once shuffled

#leave 1000 for testing
test_data = data[0:1000].T
test_Y = test_data[0]
test_X = test_data[1:n]
test_X = test_X / 255

#rest is for training
train_data = data[1000:m].T
train_Y = train_data[0]
train_X = train_data[1:n]
train_X = train_X / 255

In [7]:
print(train_X.shape)
print(train_Y.shape)

(784, 371450)
(371450,)


In [8]:
def init():
    #X -> (784, 371450)
    #Hidden Layer Depth of 20(trust), 25 output nodes
    W1 = np.random.rand(20, 784) - 0.5
    #X -> (20, 371450)
    B1 = np.random.rand(20, 1) - 0.5
    #X -> (20, 371450)
    W2 = np.random.rand(26, 20) - 0.5
    #X -> (25, 371450)
    B2 = np.random.rand(26, 1) - 0.5
    #X -> (25, 371450)
    return W1, B1, W2, B2
def softmax(X):
    A = np.exp(X) / sum(np.exp(X))
    return A
def leaky_relu(X):
    #leaky relu with alpha set as 0.01
    return np.maximum(0.01 * X, X)
def rev_relu(Z):
    #derivative of leaky relu function
    alpha = 0.01
    return np.where(Z>0, 1, alpha)

def forward(X, W1, B1, W2, B2):
    Z1 = W1.dot(X) + B1
    A1 = leaky_relu(Z1)
    Z2 = W2.dot(A1) + B2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward(X, Y, W1, B1, W2, B2, Z1, A1, Z2, A2, LR):
    #Calculate derivatives
    labels = one_hot(Y)
    dz2 = A2 - labels
    dw2 = (1 / m) * dz2.dot(A1.T)
    db2 = (1 / m) * np.sum(dz2)
    dz1 = W2.T.dot(dz2) * rev_relu(Z1)
    dw1 = (1 / m) * dz1.dot(X.T)
    db1 = (1 / m) * np.sum(dz1)
    
    #Update weights/biases
    W1 = W1 - (LR * dw1)
    B1 = B1 - (LR * db1)
    W2 = W2 - (LR * dw2)
    B2 = B2 - (LR * db2)
    
    #return new weights/biases
    return W1, B1, W2, B2

def learn(epochRate, X, Y, LR):
    W1, B1, W2, B2 = init()
    for i in range(epochRate):
        Z1, A1, Z2, A2 = forward(X, W1, B1, W2, B2)
        W1, B1, W2, B2 = backward(X, Y, W1, B1, W2, B2, Z1, A1, Z2, A2, LR)
        if (i % (0.1 * epochRate) == 0):
            print(f"Epoch {i} Completed.")
    return W1, B1, W2, B2

In [9]:
W1, B1, W2, B2 = learn(1000, train_X, train_Y, 1)

Epoch 0 Completed.
Epoch 100 Completed.
Epoch 200 Completed.
Epoch 300 Completed.
Epoch 400 Completed.
Epoch 500 Completed.
Epoch 600 Completed.
Epoch 700 Completed.
Epoch 800 Completed.
Epoch 900 Completed.


In [10]:
#To show training
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    #get accuracy
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward(X, W1, b1, W2, b2)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = train_X[:, index, None]
    prediction = make_predictions(train_X[:, index, None], W1, b1, W2, b2)
    label = train_Y[index]
    return prediction, label

In [11]:
count = 0
maxNum = 371450
for i in range(maxNum):
    pred, lab = test_prediction(i, W1, B1, W2, B2)
    if pred == lab:
        count = count + 1
print("Accuracy")
print(count / maxNum)

Accuracy
0.8845012787723785


In [12]:
dev_predictions = make_predictions(test_X, W1, B1, W2, B2)
get_accuracy(dev_predictions, test_Y)

[ 2 20 18 14 23 18 13 21 14 25 14 14 23 15  2 19  1 14  7  4 12 13 17 15
 14 20 17 13  9 22 13 14 22  3 17 13 11 23 14 17 15  6 18 25 14  3 15 18
  2  3 19 16 14 24 20 18 20  5 13 12 11 18 23 20 21 18 14 25 22 15  0 22
 14 19 20  4 18 13 12 20  2 13 13  2 20 11  3 19 12 20  0 11 14  6 18  4
 20 22 18 21  2 19  0  7  0  0 15 25 13 14 19 19  9  2  1  2  2  2 15 14
 25 25 12  0 12  1 20 18 18  0 24 25 13 24  0 18  0  9 18 19 20  9  7 14
 11 19  3  2 14 18 11 14 17 14 12 20 17 17 12 18 17  4 14 14 13 11  0 22
  0 15 13  2 14 14 13  2 20 18  2 14 24 15  3 20 18  1 11 25 20 20  0 16
 20 14 19 11  2 13 25 20 19 17  2 14 24 14 23 14 18  3  5 12 18 12 10  4
 14 24 18 22 14 24 14 12 18  2 19 15 20 14  4 18 21 18 18 18 14 18 19 18
 13  2 12  1 15 13 18  9  4  0 19  9  0 21  0  4 22 10 15 20 18 21 13  2
 19  9 23 20 11  5 11 14 14 14 18 14 13 15 14 14  2 13 20 18 23  1 18  4
 19  2  1 13 13 14 18 18 14 19 13 18  7  2 13 14 16  8 14 17 18 19 18 19
  2 11  2 14  1 20 25 20 14  2  4 14 14 20 18 23 14

0.879

About 87% accuracy on data
Decent for a 2 layer Network (1 hidden Layer, depth of 20)